# Setup 

In [4]:
#!pip install git+https://github.com/rit-git/labeler-client.git@v1.0.2

In [9]:
from labeler_client import Service, Project
import pandas as pd
import os

In [ ]:
%env TOKEN=...

# Connecting to project

In [31]:
# connecting to project
sv = Service(project='demo',token=os.environ['TOKEN'])

# Updating schema

In [16]:
sv.get_schemas().set_schemas({
    'label_schema': [
        {
            "name": "sentiment",
            "level": "record", 
            "options": [
                { "value": "pos", "text": "positive" },
                { "value": "neg", "text": "negative" },
                { "value": "neu", "text": "neutral" },
                
            ]
        },
        # adding a span-level label
                {
            "name": "sp",
            "level": "span_ch", 
            "options": [
                { "value": "pos", "text": "positive" },
                { "value": "neg", "text": "negative" },
            ]
        }
    ]
})
sv.get_schemas().value(active=True)

[{'active': True,
  'created_on': 1663170874537,
  'schemas': {'label_schema': [{'level': 'record',
     'name': 'sentiment',
     'options': [{'text': 'positive', 'value': 'pos'},
      {'text': 'negative', 'value': 'neg'},
      {'text': 'neutral', 'value': 'neu'}]},
    {'level': 'span_ch',
     'name': 'sp',
     'options': [{'text': 'positive', 'value': 'pos'},
      {'text': 'negative', 'value': 'neg'}]}]},
  'uuid': 'ceaa6f3d-d949-4eda-a11f-7f5ce035b6a8'}]

In [17]:
# get schema history
sv.get_schemas().get_history()

[{'active': False,
  'created_on': 1663170742093,
  'schemas': {'label_schema': [{'level': 'record',
     'name': 'sentiment',
     'options': [{'text': 'positive', 'value': 'pos'},
      {'text': 'negative', 'value': 'neg'}]},
    {'level': 'span_ch',
     'name': 'sp',
     'options': [{'text': 'positive', 'value': 'pos'},
      {'text': 'negative', 'value': 'neg'}]}]},
  'uuid': '6332fa12-2db3-4374-899e-73a4a1a22430'},
 {'active': False,
  'created_on': 1663168856667,
  'schemas': {'label_schema': [{'level': 'record',
     'name': 'sentiment',
     'options': [{'text': 'positive', 'value': 'pos'},
      {'text': 'negative', 'value': 'neg'}]}]},
  'uuid': '523f1256-820b-4b83-a55f-07cf5330c12d'}]

# Loading data with metadata

In [20]:
#load data
df = pd.read_csv("tweets.csv").loc[:100]
sv.import_data_df(df, column_mapping={'id':'id',
                                      'content':'content',
                                      "metadata":'pseudo_label'})
s1= sv.search(keyword='',
                  limit=10,
                  start=0, 
                  meta_names=['pseudo_label'])
s1.show()

show(7fd8ed19a040, self=<labeler_ui.widgets.Annotation.Annotation object at 0x7fd8eaf71280>)

LayoutWidget(Layout(show(7fd8ed19a040, self=<labeler_ui.widgets.Annotation.Annotation object at 0x7fd8eaf71280…

# Utilizing metadata 
## Example 1 : customized embedding

In [52]:
# Example 1, adding sentence-bert embedding.
# metadata searches.
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
# set metadata generation function 
sv.set_metadata("bert-embedding",lambda x: list(model.encode(x).astype(float)), 500)

/opt/miniconda3/envs/labeler/lib/python3.8/site-packages/huggingface_hub/file_download.py:560: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(
Metadata batches processed:: 100%|███████████████████████████████████████| 1/1 [00:06<00:00,  6.35s/it]

Finished setting metadata bert-embedding for 201 data records


In [37]:
s3 = sv.search(keyword='delay', limit=3, start=0)
s3.show()

show(7fd8dd307940, self=<labeler_ui.widgets.Annotation.Annotation object at 0x7fd8dd2fbfd0>)

LayoutWidget(Layout(show(7fd8dd307940, self=<labeler_ui.widgets.Annotation.Annotation object at 0x7fd8dd2fbfd0…

In [39]:
s4 = s3.suggest_similar('bert-embedding', limit=4)# needs to provide a valid meta_name
s4.show()

show(7fd8c85ca400, self=<labeler_ui.widgets.Annotation.Annotation object at 0x7fd8c9b91190>)

LayoutWidget(Layout(show(7fd8c85ca400, self=<labeler_ui.widgets.Annotation.Annotation object at 0x7fd8c9b91190…

## Example 2: showing hashtags as context

In [60]:
# user defined function
def extract_hashtags(text):
    hashtag_list = []
    for word in text.split():
        if word[0] == '#':
            hashtag_list.append(word[:])
    # widget can render markdown text
    return ''.join(['- {}\n'.format(x) for x in hashtag_list])

# apply metadata to project
sv.set_metadata("hashtag",lambda x: extract_hashtags(x), 500)

Metadata batches processed:: 100%|███████████████████████████████████████| 1/1 [00:02<00:00,  2.74s/it]

Finished setting metadata hashtag for 201 data records


In [58]:
s1= sv.search(keyword='',
                  limit=50,
                  start=0, 
                  meta_names=['hashtag'])
s1.show()

show(7fd8dd307e20, self=<labeler_ui.widgets.Annotation.Annotation object at 0x7fd8c8580130>)

LayoutWidget(Layout(show(7fd8dd307e20, self=<labeler_ui.widgets.Annotation.Annotation object at 0x7fd8c8580130…

# Explorating 

In [17]:
s1 = sv.get_subset(keyword='delay', limit=50, start=0)
s1.show({'view':'table'})

show(7ffd237fb280, self=<labeler_ui.widgets.Annotation.Annotation object at 0x7ffd2388e820>)

LayoutWidget(Layout(show(7ffd237fb280, self=<labeler_ui.widgets.Annotation.Annotation object at 0x7ffd2388e820…

# Anlysis Dashboard

In [61]:
#Analysis
from labeler_ui import Dashboard
dash_wg = Dashboard(sv,{'height':500})
dash_wg.show()


show(7fd8ed19afa0, self=<labeler_ui.widgets.Dashboard.Dashboard object at 0x7fd8c857f4f0>)

LayoutWidget(Layout(show(7fd8ed19afa0, self=<labeler_ui.widgets.Dashboard.Dashboard object at 0x7fd8c857f4f0>)…

In [62]:
sv.export()

,data_id,content,annotator,label_name,label_value
0,567862940348579840,@united I've been doing this for 15 years and ...,00hUwRoyemQtnV7I91R2TpuIcsC3,sentiment,[neg]
1,567863197341847616,"@SouthwestAir Yeah, we figured it out. Thanks.",00hUwRoyemQtnV7I91R2TpuIcsC3,sentiment,[pos]
2,567863445422546880,@SouthwestAir On my flight today from RSW to G...,00hUwRoyemQtnV7I91R2TpuIcsC3,sentiment,[pos]
3,567871168214106112,@USAirways Your whole handling of this was a j...,00hUwRoyemQtnV7I91R2TpuIcsC3,sentiment,[neg]
4,567860325179072512,@SouthwestAir took our BOS to BWI plane for a ...,00hUwRoyemQtnV7I91R2TpuIcsC3,sentiment,[neg]
5,567861727833894912,@united mechanical issues causes delay and con...,00hUwRoyemQtnV7I91R2TpuIcsC3,sentiment,[neg]
6,567864523585118208,@USAirways delayed my flight 3 times before Ca...,00hUwRoyemQtnV7I91R2TpuIcsC3,sentiment,[neg]
